In [5]:
import numpy as np
import os
from multiprocessing import Process
import subprocess
import pandas as pd
import json
import random

import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import models

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'true'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

DEBUG = False

In [6]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
files = '/home/joren/Coding/cryptodata/Normalized_labelled/'
csv_files = '/home/joren/Coding/cryptodata/1MIN/'
modelfiles = ['model_small_10']
frame_sizes = [320]
data_length = 230000

In [7]:
def filenames(dir):
    filenames = []
    for file in os.listdir(dir):
        filename = os.fsdecode(file)
        filenames.append(filename)
    random.shuffle(filenames)
    return filenames

field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data, dtype=np.float16)

# train-test split
def df_split(df, frame_size):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_test = []
    y_test = []
    for i in range(frame_size, X.shape[0]): 
        X_test.append(X[i-frame_size:i])
        y_test.append(Y[i])
    X_test, y_test = np.array(X_test, dtype=np.float16), np.array(y_test, dtype=np.float16)

    return X_test, y_test

In [8]:
#normal extra long
for modelfile_index in range(len(modelfiles)):
    modelfile = './models/'+modelfiles[modelfile_index]
    frame_size = frame_sizes[modelfile_index]

    for file in filenames(files):
        ## prepare data for prediction
        data = read_bin_full_file(files+file)
        df2 = pd.DataFrame(data, columns=column_names)

        for x in range(round(len(df2)/data_length)):
            if (x+1)*data_length >= len(df2):
                df = df2.iloc[x*data_length:-1]
                subprocess.call(["/bin/python3.9" ,"predict.py", f"{modelfile}", f"{x*data_length}", "-1", f"{frame_size}", f"{files+file}", f"{file}"]) 
            else:
                df = df2.iloc[x*data_length:(x+1)*data_length]
                subprocess.call(["/bin/python3.9" ,"predict.py", f"{modelfile}", f"{x*data_length}", f"{(x+1)*data_length}", f"{frame_size}", f"{files+file}",  f"{file}"]) 
                # subprocess.call(["/bin/python3.9" ,f"predict.py {modelfile} {x*data_length} {(x+1)*data_length} {frame_size} {files+files}"])                  

2022-02-17 11:07:05.444344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:07:05.472299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:07:05.472472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:07:05.665623: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

{'percentage': 13.967443936717402, 'negative': False, 'length_in_hours': 3828.0, 'trades': 57489}


2022-02-17 11:17:28.050599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:17:28.076267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:17:28.076441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:17:28.250375: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

{'percentage': 15.050864750974837, 'negative': False, 'length_in_hours': 3833.3333333333335, 'trades': 46244}


2022-02-17 11:27:41.357119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:27:41.383191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:27:41.383368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 11:27:41.556925: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

...

{'percentage': 23.202886930785276, 'negative': False, 'length_in_hours': 3833.3333333333335, 'trades': 21169}


2022-02-17 16:09:08.776925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:09:08.812057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:09:08.812246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:09:09.058035: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

{'percentage': 17.571498805606396, 'negative': False, 'length_in_hours': 3078.6666666666665, 'trades': 23611}


2022-02-17 16:18:30.855498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:18:30.904651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:18:30.904860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-17 16:18:31.200124: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

KeyboardInterrupt: 

In [ ]:
for modelfile_index in range(len(modelfiles)):
    modelfile = './models/'+modelfiles[modelfile_index]
    frame_size = frame_sizes[modelfile_index]

    for file in filenames(files):
        ## prepare data for prediction
        data = read_bin_full_file(files+file)
        df2 = pd.DataFrame(data, columns=column_names)

        for x in range(round(len(df2)/data_length)):
            if (x+1)*data_length >= len(df2):
                df = df2.iloc[x*data_length:-1]
            else:
                df = df2.iloc[x*data_length:(x+1)*data_length]

            if len(df) < frame_size*5:
                continue

            X_test, y_test = df_split(df, frame_size)

            # calculate(X_test, df, modelfile)

            # load model
            # model = models.load_model(modelfile)
            # P = Process(target = calculate, args=(X_test, df, model))
            # P.start()
            # P.join()

            ### SEND TO FILE
            # modelfile
            # start_index
            # stop_index
            # frame_size

In [ ]:
# PIRAMIDDING (work in progress)
for modelfile_index in range(len(modelfiles)):
    modelfile = './models/'+modelfiles[modelfile_index]
    frame_size = frame_sizes[modelfile_index]

    # load model
    model = models.load_model(modelfile)

    i = 0
    total_percentage_raw = 0
    negatives = []
    length = 0

    for file in filenames(files):
        ## prepare data for prediction
        data = read_bin_full_file(files+file)
        df2 = pd.DataFrame(data, columns=column_names)
        startloc = round(len(df2)*0.97)
        df = df2.iloc[startloc:]
        if len(df) < frame_size*5:
            continue
        i += 1

        print(df2.tail())

        X_test, y_test = df_split(df, frame_size)

        ## predict
        y_pred = np.array(model.predict(X_test), dtype=np.float16)

        y_pred = np.argmax(y_pred, axis=-1)
        y_pred = y_pred.flatten()

        df = df.iloc[frame_size:]
        length += len(df)/60
        df['predictions'] = np.array(y_pred)

        print(df.tail())

        # calculate profit/loss
        df.drop(columns=['open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target'], inplace=True)
        df.set_index('timestamp')

        print(df.tail())
        
        df2 = pd.read_csv(csv_files+file[:-4]+'.csv')
        df2.drop(columns=['open','high','low'], inplace=True)
        df2.set_index('event_time')

        print(df2.tail())
        
        df.join(df2)

        print(df.tail())

        temp_percentage = 0
        percentage = 0
        position = 0

        # for index, row in df.iterrows():

        #     if position == 1:
        #         percentage += row['close']

        #     if row['predictions'] == 1:
        #         position = 1
        #     elif row['predictions'] == 2:
        #         position = 0
        #         total_percentage_raw += percentage
        #         temp_percentage += percentage
        #         percentage = 0

        # if temp_percentage < 0:
        #     negatives.append({
        #         'file': file,
        #         'result': temp_percentage
        #     })
        #     print(f"negatives: {negatives}")

        # dictionary ={
        #     "model" : modelfile,
        #     "files_processed": i,
        #     "average_percentage" : total_percentage_raw/i,
        #     "negative_results": negatives,
        #     "average_length_in_hours": length/i
        # }

        # # new_object = json.dumps(dictionary, indent = 4)
        # # json_object['results'].append(dictionary)
        # json_object = json.dumps(dictionary, indent = 4)
        # with open(f"test_results/{modelfiles[modelfile_index]}_result.json", "a") as outfile:
        #     outfile.write(json_object)
        

        timestamp      open     close      high       low    volume     adosc  \
583419        inf -0.000753 -0.001129 -0.001128 -0.000753 -0.388184  0.327393   
583420        inf -0.001128 -0.001503 -0.001878 -0.000753 -0.140259  0.381592   
583421        inf -0.001503 -0.000751  0.000000 -0.001503  6.957031  0.483643   
583422        inf -0.000376 -0.001500 -0.001874  0.000000 -0.782715  0.717285   
583423        inf -0.001500  0.000375  0.000375 -0.001126  0.982422  1.668945   

             atr      macd  macd_signal  macd_hist       mfi  upper_band  \
583419 -0.720215 -0.287842    -0.178467   2.683594  0.004787    0.744141   
583420 -1.875000 -1.448242    -0.538086  -0.918457  0.004498   -0.357422   
583421 -0.486572  1.949219    -0.543945   1.373047  0.003574    0.459229   
583422  7.093750  2.265625    -0.774414  -2.691406  0.004051   -0.141357   
583423 -0.673340 -1.968750    -3.369141  -0.805176  0.004906    1.635742   

        middle_band  lower_band       rsi  difference_lo

KeyboardInterrupt: 

In [ ]:
# normal
for modelfile_index in range(len(modelfiles)):
    modelfile = './models/'+modelfiles[modelfile_index]
    frame_size = frame_sizes[modelfile_index]

    # load model
    model = models.load_model(modelfile)

    i = 0
    total_percentage_raw = 0
    negatives = []
    length = 0

    for file in filenames(files):
        ## prepare data for prediction
        data = read_bin_full_file(files+file)
        df2 = pd.DataFrame(data, columns=column_names)
        # df2.fillna(0, inplace=True)
        startloc = round(len(df2)*0.98)
        df = df2.iloc[startloc:]
        if len(df) < frame_size*5:
            continue
        i += 1


        X_test, y_test = df_split(df, frame_size)

        ## predict
        y_pred = np.array(model.predict(X_test), dtype=np.float16)

        y_pred = np.argmax(y_pred, axis=-1)
        y_pred = y_pred.flatten()

        df = df.iloc[frame_size:]
        length += len(df)/60
        df['predictions'] = np.array(y_pred)

        # calculate profit/loss
        temp_percentage = 0
        percentage = 0
        position = 0

        for index, row in df.iterrows():

            if position == 1:
                percentage += row['close']

            if row['predictions'] == 1:
                position = 1
            elif row['predictions'] == 2:
                position = 0
                total_percentage_raw += percentage
                temp_percentage += percentage
                percentage = 0

        if temp_percentage < 0:
            negatives.append({
                'file': file,
                'result': temp_percentage
            })
            print(f"negatives: {negatives}")

        dictionary ={
            "model" : modelfile,
            "files_processed": i,
            "average_percentage" : total_percentage_raw/i,
            "negative_results": negatives,
            "average_length_in_hours": length/i
        }

        # new_object = json.dumps(dictionary, indent = 4)
        # json_object['results'].append(dictionary)
        json_object = json.dumps(dictionary, indent = 4)
        with open(f"test_results/{modelfiles[modelfile_index]}_result.json", "a") as outfile:
            outfile.write(json_object)
        